#Итоговый проект

===============================
# 1️⃣ УСТАНОВКА ЗАВИСИМОСТЕЙ
===============================

In [1]:

!pip uninstall -y openai whisper anyio -q
!pip install -q anyio>=4.9.0
!pip install -q openai>=1.6.0 openai-whisper ffmpeg-python python-docx

===============================
#2️⃣ ИМПОРТ БИБЛИОТЕК
===============================

In [2]:

import os
import time
import whisper
from datetime import datetime
from google.colab import files, userdata
from docx import Document
from openai import OpenAI
from IPython.display import display, HTML

===============================
#3️⃣ КЛАСС NeuroSecretaryColab
===============================

In [3]:

class NeuroSecretaryColab:
    """
    Нейро-секретарь для:
    - транскрибации аудио/видео (Whisper)
    - анализа текста (GPT)
    - формирования протокола
    - экспорта в Word
    """

    def __init__(self, api_key: str | None = None):
        self.api_key = api_key or userdata.get("OPENAI_API_KEY")

        if not self.api_key:
            raise ValueError(
                "❌ OpenAI API ключ не найден. "
                "Добавьте его в Colab → Secrets → OPENAI_API_KEY"
            )

        self.client = OpenAI(api_key=self.api_key)
        print(f"✅ NeuroSecretaryColab инициализирован (длина ключа {len(self.api_key)})")

        print("🤖 Загружаем модель Whisper...")
        self.whisper_model = whisper.load_model("base")
        print("✅ Модель Whisper загружена")

    # ---------- ВСПОМОГАТЕЛЬНО ----------
    def _format_time(self, seconds: float) -> str:
        m = int(seconds // 60)
        s = int(seconds % 60)
        return f"{m:02d}:{s:02d}"

    # ---------- ТРАНСКРИБАЦИЯ ----------
    def transcribe_file(self, file_path: str):
        print(f"🎤 Транскрибация файла: {file_path}")

        result = self.whisper_model.transcribe(
            file_path,
            language="ru",
            fp16=False,
            verbose=False
        )

        text = ""
        for seg in result.get("segments", []):
            start = self._format_time(seg["start"])
            end = self._format_time(seg["end"])
            content = seg["text"].strip()
            if content:
                text += f"[{start}–{end}] {content}\n"

        print(f"✅ Транскрибация завершена ({len(text)} символов)")
        return text.strip()

    # ---------- GPT-ПРОТОКОЛ ----------
    def generate_protocol(self, transcription: str) -> str:
        print("🧠 Анализируем транскрипцию с помощью GPT...")

        system_prompt = (
            "Ты — профессиональный секретарь, который составляет "
            "структурированные протоколы деловых встреч."
        )

        user_prompt = f"""
Проанализируй транскрипцию совещания и составь ПРОТОКОЛ.

Структура:
1. Основная информация
2. Участники
3. Повестка дня
4. Ключевые обсуждения
5. Принятые решения
6. Задачи и ответственные

Транскрипция:
{transcription[:8000]}
"""

        response = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,
            max_tokens=1500
        )

        protocol = response.choices[0].message.content
        print("✅ Протокол создан")
        return protocol

    # ---------- WORD ----------
    def create_word_report(
        self,
        topic: str,
        description: str,
        transcription: str,
        protocol: str,
        file_name: str = "report.docx"
    ):
        doc = Document()
        doc.add_heading(topic, 0)
        doc.add_paragraph(description)

        doc.add_heading("Транскрипция", level=1)
        doc.add_paragraph(transcription)

        doc.add_heading("Протокол", level=1)
        doc.add_paragraph(protocol)

        doc.add_paragraph(
            f"Отчет сгенерирован: {datetime.now().strftime('%d.%m.%Y %H:%M')}"
        )

        doc.save(file_name)
        print(f"📄 Word-файл создан: {file_name}")
        return file_name

===============================
#4️⃣ ПРОСТОЙ DARK UI
===============================

In [4]:

def show_ui():
    display(HTML("""
    <style>
        body { background:#0f0f0f; color:#eee; font-family:Arial; }
        .box {
            border:2px solid #e6b7b2;
            border-radius:12px;
            padding:20px;
            background:#1a1a1a;
            margin:10px 0;
        }
        h2 { color:#f0c0b0; }
    </style>
    <div class="box">
        <h2>📂 Загрузка файла встречи</h2>
        <p>Поддержка: mp3 / wav / mp4</p>
    </div>
    """))


===============================
#5️⃣ ГЛАВНАЯ ФУНКЦИЯ ЗАПУСКА
===============================

In [5]:

def run_neurosecretary():
    ns = NeuroSecretaryColab()

    show_ui()
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]

    transcript = ns.transcribe_file(file_path)

    protocol = ns.generate_protocol(transcript)

    report = ns.create_word_report(
        topic="Протокол совещания",
        description="Автоматически сгенерировано нейро-секретарём.",
        transcription=transcript,
        protocol=protocol
    )

    files.download(report)

    print("\n🎉 ГОТОВО!")
    print("— Транскрипция получена")
    print("— Протокол сформирован")
    print("— Word-файл скачан")

In [5]:
def run_neurosecretary():
    ns = NeuroSecretaryColab()

    while True:
        show_ui()
        uploaded = files.upload()

        if not uploaded:
            print("❌ Файл не выбран")
            break

        file_path = list(uploaded.keys())[0]

        transcript = ns.transcribe_file(file_path)
        protocol = ns.generate_protocol(transcript)

        report = ns.create_word_report(
            topic="Протокол совещания",
            description="Автоматически сгенерировано нейро-секретарём.",
            transcription=transcript,
            protocol=protocol,
            file_name=f"report_{int(time.time())}.docx"
        )

        files.download(report)

        answer = input("\n👉 Загрузить ещё один файл? (y/n): ").lower()
        if answer != "y":
            print("👋 Работа завершена")
            break


===============================
#6️⃣ 🔥 ТЕСТОВАЯ ЯЧЕЙКА (ДЛЯ СДАЧИ)
===============================

In [6]:

run_neurosecretary()


✅ NeuroSecretaryColab инициализирован (длина ключа 164)
🤖 Загружаем модель Whisper...


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 256MiB/s]


✅ Модель Whisper загружена


Saving Что лучше С# или С++ 2023 #шортс #технологии #программирование.mp4 to Что лучше С# или С++ 2023 #шортс #технологии #программирование.mp4
🎤 Транскрибация файла: Что лучше С# или С++ 2023 #шортс #технологии #программирование.mp4


100%|██████████| 1890/1890 [00:07<00:00, 241.56frames/s]


...

100%|██████████| 77515/77515 [03:33<00:00, 363.23frames/s]


✅ Транскрибация завершена (14349 символов)
🧠 Анализируем транскрипцию с помощью GPT...
✅ Протокол создан
📄 Word-файл создан: report_1768758035.docx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


👉 Загрузить ещё один файл? (y/n): n
👋 Работа завершена
